<a href="https://colab.research.google.com/github/wr0124/Learning_essential/blob/main/AnimateDiff_motion_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.5 MB/s eta 0:00:00


In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange

In [3]:
x=torch.randn(2, 3, 4, 2, 2)
print(x)

tensor([[[[[ 1.4563, -0.1745],
           [-1.9305, -0.6525]],

          [[-0.2448, -0.5148],
           [-1.0070, -1.1509]],

          [[ 0.8268,  0.8601],
           [ 1.0506,  2.1072]],

          [[-0.0615, -0.5769],
           [ 0.4600, -0.2355]]],


         [[[-1.6677,  0.6653],
           [ 0.7922,  1.5480]],

          [[ 2.3729, -0.0524],
           [-1.1576,  1.6254]],

          [[ 0.9990, -0.6284],
           [-0.4289, -0.8202]],

          [[ 0.4457, -0.5164],
           [-0.7976,  1.1081]]],


         [[[ 1.7992, -0.9455],
           [ 0.1406, -0.1120]],

          [[ 0.3925, -0.3140],
           [ 0.0042, -0.0285]],

          [[-0.1334,  1.2930],
           [ 0.7025,  0.6880]],

          [[-1.8323,  0.1044],
           [-0.6214, -0.1715]]]],



        [[[[-0.5735, -0.0211],
           [-0.9169, -2.7385]],

          [[-1.2171,  0.8684],
           [ 0.8424, -0.2886]],

          [[ 1.0189, -0.2857],
           [ 0.0287,  1.4488]],

          [[ 1.1815, -0.3659],
 

In [4]:

class InflatedConv3d(nn.Conv2d):
    def forward(self, x):
        print(x.shape)
        video_length = x.shape[2]
        print(video_length)
        x = rearrange(x, "b c f h w -> (b f) c h w")
        print(x.shape)
        x = super().forward(x)
        print(x.shape)
        x = rearrange(x, "(b f) c h w -> b c f h w", f=video_length)
        print(x.shape)
        return x

In [7]:
conv_in = InflatedConv3d(3, 32, kernel_size=3, padding=(1, 1))
print(conv_in)
output_tensor = conv_in (x)

InflatedConv3d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
torch.Size([2, 3, 4, 2, 2])
4
torch.Size([8, 3, 2, 2])
torch.Size([8, 32, 2, 2])
torch.Size([2, 32, 4, 2, 2])


#1 PositionalEncoding

In [8]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(
        self,
        d_model,
        dropout = 0.,
        max_len = 24
    ):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        print("position",position)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        print("div_term", div_term )
        pe = torch.zeros(1, max_len, d_model)
        print("pe", pe)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        print("pe", pe)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        print("pe", pe)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [9]:
# Example usage
import torch

# Assuming d_model=512, max_len=24
pos_enc = PositionalEncoding(d_model=2, max_len=24)

input_tensor = torch.randn(1, 10, 2)  # Input tensor of shape (batch_size=1, sequence_length=100, d_model=512)
print("before add into pos_enc ", input_tensor)
output_tensor = pos_enc(input_tensor)   # Output tensor after adding positional encoding

print("after add into pos_enc",output_tensor)  # Output shape will be (1, 100, 512)

position tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21],
        [22],
        [23]])
div_term tensor([1.])
pe tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])
pe tensor([[[ 0.0000,  0.0000],
         [ 0.8415,  0.0000],
         [ 0.9093,  0.0000],
         [ 0.1411,  0.0000],
         [-0.7568,  0.0000],
         [-0.9589,  0.0

# 2 CorssAttention

In [10]:
from inspect import isfunction
def exists(val):
    return val is not None

def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d

# Example 1
value = 42
result = default(value, 0)
print(result)  # Output: 42

# Example 2
value = None
result = default(value, "default")
print(result)  # Output: "default"

# Example 3
value = ""
result = default(value, "default")
print(result)  # Output: ""

# Example 4
value = []
result = default(value, [1, 2, 3])
print(result)  # Output: []

# Example 5
value = {}
result = default(value, {"key": "value"})
print(result)  # Output: {}

# Example 6
value = None
result = default(value, lambda: "computed_default")
print(result)  # Output: "computed_default"


42
default

[]
{}
computed_default


In [ ]:
##https://github.com/huggingface/diffusers_all/blob/main/src/diffusers/models/attention.py


class CrossAttention(nn.Module):
    def __init__(self, query_dim, context_dim=None, heads=8, dim_head=64, dropout=0.0):
        super().__init__()
        inner_dim = dim_head * heads
        context_dim = default(context_dim, query_dim)

        self.scale = dim_head**-0.5
        self.heads = heads

        self.to_q = nn.Linear(query_dim, inner_dim, bias=False)
        self.to_k = nn.Linear(context_dim, inner_dim, bias=False)
        self.to_v = nn.Linear(context_dim, inner_dim, bias=False)

        self.to_out = nn.Sequential(nn.Linear(inner_dim, query_dim), nn.Dropout(dropout))

    def reshape_heads_to_batch_dim(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.heads
        tensor = tensor.reshape(batch_size, seq_len, head_size, dim // head_size)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size * head_size, seq_len, dim // head_size)
        return tensor

    def reshape_batch_dim_to_heads(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.heads
        tensor = tensor.reshape(batch_size // head_size, head_size, seq_len, dim)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size // head_size, seq_len, dim * head_size)
        return tensor

    def forward(self, x, context=None, mask=None):
        batch_size, sequence_length, dim = x.shape

        h = self.heads

        q = self.to_q(x)
        context = default(context, x)
        k = self.to_k(context)
        v = self.to_v(context)

        q = self.reshape_heads_to_batch_dim(q)
        k = self.reshape_heads_to_batch_dim(k)
        v = self.reshape_heads_to_batch_dim(v)

        sim = torch.einsum("b i d, b j d -> b i j", q, k) * self.scale

        if exists(mask):
            mask = mask.reshape(batch_size, -1)
            max_neg_value = -torch.finfo(sim.dtype).max
            mask = mask[:, None, :].repeat(h, 1, 1)
            sim.masked_fill_(~mask, max_neg_value)

        # attention, what we cannot get enough of
        attn = sim.softmax(dim=-1)

        out = torch.einsum("b i j, b j d -> b i d", attn, v)
        out = self.reshape_batch_dim_to_heads(out)
        return self.to_out(out)

